In [3]:
# convert folder of csv format
#ID,LENGTH,DIR,A,B,ROADNAME,OTHERNAME,PROJECTNAM,YEAR,LEVEL_110,CLASS_110,LANES_110,ROW,S0,ALPHA,BETA,NMT_110,NTK_110,NBUS_110,COUNTY,MRTXFERPEN,ETAG
# "4",0.30,"2","17252","10118",台2,登輝大道,,"104","3","21","3",,"50",1.1507,3.7967,"0","0","0","2","0",0.0000
# to dat format
# tail.node	head.node	capacity..veh.h.	length..miles.	fftt.min.	Power	B	speed.limit..mph.
# 1	54632	99999	0	2.19029	0.9	3.6	NA

import pandas as pd

In [12]:
# Network

cap_dict={56: 1300, 28: 1300, 25: 1450, 76: 3320, 46: 770, 21: 3920, 16: 1650, 54: 1170, 47: 910, 64: 860, 38: 1170, 22: 1170, 63: 810, 65: 910, 2: 3870, 3: 2300, 62: 770, 1: 8300, 4: 2070, 27: 5880, 70: 1300, 39: 3140, 36: 3490, 15: 1860, 17: 2949, 33: 1950, 43: 5880, 49: 2949, 32: 2490, 66: 2490, 51: 1450, 69: 1950, 30: 770, 71: 1450, 37: 3920, 50: 1300, 31: 1860, 35: 3920, 45: 1250, 41: 3920, 19: 3920, 12: 3920, 7: 4100, 68: 1850, 74: 1170, 14: 1570, 42: 5241, 78: 1300, 9: 2050, 67: 2631, 8: 3690, 10: 1850, 81: 5880, 79: 5880, 26: 5241, 40: 1300, 20: 3490, 60: 1070, 23: 3140, 80: 3490, 34: 1300, 48: 2490, 82: 1070, 59: 5880, 58: 1750, 83: 3741, 13: 3530, 24: 1300, 5: 3600, 57: 1450, 18: 1300, 29: 2500, 11: 3200, 55: 4710, 73: 5880, 77: 3320, 52: 1750, 72: 1750, 6: 5100, 99: 9999}

cap = pd.read_csv("Y110_Net_assigned.csv")
link_cap={}
for index, row in cap.iterrows():
    link_cap[(row["A"], row["B"])] = row["CAPACITY"]

input_file = 'Y110_TRTS4S_Net.csv'
output_file = 'network.dat'

df = pd.read_csv(input_file)
# if there is '', then replace with 0
df_out = pd.DataFrame(columns=['origin','dest','capacity','length','fft','alpha','beta','speedLimit'])
df.fillna(0, inplace=True)
rows = []

count=0
ncount=0

for index, row in df.iterrows():
    if row["LEVEL_110"] in [1,2,3,4,5,6,11] and row["CLASS_110"] not in [i for i in range(84, 99)]+[61]:
        count+=1
        # if DIR=1, then A is origin, B is dest
        # if DIR=-1, then A is dest, B is origin
        # if DIR=2, then the road is bidirectional, so we need to add two rows
        if row["DIR"] == 1 or row["DIR"] == 2:
            # print(row["ID"], row["DIR"], row["A"], row["B"])
            origin = row["A"]
            dest = row["B"]
        elif row["DIR"] == -1:
            # print(row["ID"], row["DIR"], row["A"], row["B"])
            origin = row["B"]
            dest = row["A"]
        if (row['A'], row['B']) in link_cap:
            capacity = link_cap[(row['A'], row['B'])]
        else:
            capacity = cap_dict[row["CLASS_110"]]
            ncount+=1
        length = float(row["LENGTH"])  or 1000
        try:
            fft = length/float(row["S0"])*60
        except:
            # if ID is not 40438, then give S0=32
            # print(row["ID"], row["S0"])
            if row["ID"] != 40438:
                fft = length/32*60
            else:
                fft = 1000
        alpha = row["ALPHA"] or 0
        beta = row["BETA"] or 0
        speedLimit = 0
        rows.append({'origin':origin,'dest':dest,'capacity':capacity,'length':length,'fft':fft,'alpha':alpha,'beta':beta,'speedLimit':speedLimit})
        if row["DIR"] == 2:
            # add the reverse direction
            rows.append({'origin':dest,'dest':origin,'capacity':capacity,'length':length,'fft':fft,'alpha':alpha,'beta':beta,'speedLimit':speedLimit})

# tail.node	head.node	capacity..veh.h.		fftt.min.			speed.limit..mph.

df_out = pd.concat([df_out, pd.DataFrame(rows)], ignore_index=True)
df_out.to_csv(output_file, index=False, sep='\t')

print("Total number of links that are not in the link capacity file: ", ncount, "out of ", count) 

Total number of links that are not in the link capacity file:  4065 out of  14431


C:\Users\Allen\AppData\Local\Temp\ipykernel_19796\1980384817.py:61: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_out = pd.concat([df_out, pd.DataFrame(rows)], ignore_index=True)


In [1]:
# OD flow
# read TRTS4S_Y110指派OD與交通分區.xlsx
# sheet_name = '110年指派OD晨峰'
# columm A,B,H,I as origin,dest,all,bus
# demand as all+bus*1.8/19

import pandas as pd
import numpy as np

input_file = 'TRTS4S_Y110指派OD與交通分區 - 複製.xlsx'
output_file = 'demand.dat'

df = pd.read_excel(input_file, sheet_name='110年指派OD晨峰')
df.fillna(0, inplace=True)
rows = []

print("Successfully read the file")

count=0

for index, row in df.iterrows():
    origin = row["I"]
    dest = row["J"]
    all = row["ALL"]
    bus = row["BUS"]
    demand = (all+bus*1.8/19)*0.5
    # if i,j > 631, do not add to the list
    if origin > 631 or dest > 631:
        continue
    rows.append({'origin':int(origin),'dest':int(dest),'demand':demand})
    if count%10000==0:
        print(count)
    count+=1

df_out = pd.DataFrame(rows)
df_out.to_csv(output_file, index=False, sep='\t')


Successfully read the file
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
